## Walkthrough provided by:
Tatman, R. (2018, March 30). Data Cleaning challenge: Handling missing values. www.kaggle.com. https://www.kaggle.com/code/rtatman/data-cleaning-challenge-handling-missing-values

Here's what we're going to do today:

* [Take a first look at the data](#Take-a-first-look-at-the-data)
* [See how many missing data points we have](#See-how-many-missing-data-points-we-have)
* [Figure out why the data is missing](#Figure-out-why-the-data-is-missing)
* [Drop missing values](#Drop-missing-values)
* [Filling in missing values](#Filling-in-missing-values)

Let's get started!

# Take a first look at the data
________

The first thing we'll need to do is load in the libraries and datasets we'll be using. For today, I'll be using a dataset of events that occured in American Football games for demonstration, and you'll be using a dataset of building permits issued in San Francisco.

> **Important!** Make sure you run this cell yourself or the rest of your code won't work!

In [ ]:
# modules we'll use
import pandas as pd
import numpy as np

# read in all our data
nfl_data = pd.read_csv("NFL Play by Play 2009-2017 (v4).csv")
sf_permits = pd.read_csv("Building_Permits.csv")

# set seed for reproducibility
np.random.seed(0) 

The first thing I do when I get a new dataset is take a look at some of it. This lets me see that it all read in correctly and get an idea of what's going on with the data. In this case, I'm looking to see if I see any missing values, which will be reprsented with `NaN` or `None`.

In [ ]:
# look at a few rows of the nfl_data file. I can see a handful of missing data already!
nfl_data.sample(5)

Yep, it looks like there's some missing values. What about in the sf_permits dataset?

In [ ]:
# your turn! Look at a couple of rows from the sf_permits dataset. Do you notice any missing data?

# your code goes here :)


# See how many missing data points we have
___

Ok, now we know that we do have some missing values. Let's see how many we have in each column. 

In [ ]:
# get the number of missing data points per column
missing_values_count = nfl_data.isnull().sum()

# look at the # of missing points in the first ten columns
missing_values_count[0:10]

That seems like a lot! It might be helpful to see what percentage of the values in our dataset were missing to give us a better sense of the scale of this problem:

- `nfl_data.shape`: This returns a tuple with the number of rows and columns in the nfl_data DataFrame.

- `np.product(nfl_data.shape)`: Multiplies the number of rows and columns to calculate the total number of cells in the DataFrame.

- `missing_values_count`: This variable likely contains the count of missing values for each column in the dataset (possibly calculated with `nfl_data.isnull().sum())`.

- .`sum()`: Sums the missing values across all columns, resulting in the total number of missing values in the DataFrame.

- `(total_missing / total_cells) * 100`: Calculates the percentage of the dataset that has missing values by dividing the number of missing values by the total number of cells and then multiplying by 100.

In [ ]:
# how many total missing values do we have?
total_cells = np.product(nfl_data.shape)
total_missing = missing_values_count.sum()

# percent of data that is missing
(total_missing/total_cells) * 100

Wow, almost a quarter of the cells in this dataset are empty! In the next step, we're going to take a closer look at some of the columns with missing values and try to figure out what might be going on with them.

In [ ]:
# your turn! Find out what percent of the sf_permits dataset is missing

# get the number of missing data points per column

# look at the # of missing points in the first ten columns

# how many total missing values do we have?

# percent of data that is missing


# Figure out why the data is missing
____
 
This is the point at which we get into the part of data science that I like to call "data intution", by which I mean "really looking at your data and trying to figure out why it is the way it is and how that will affect your analysis". It can be a frustrating part of data science, especially if you're newer to the field and don't have a lot of experience. For dealing with missing values, you'll need to use your intution to figure out why the value is missing. One of the most important question you can ask yourself to help figure this out is this:

> **Is this value missing becuase it wasn't recorded or becuase it dosen't exist?**

If a value is missing becuase it doens't exist (like the height of the oldest child of someone who doesn't have any children) then it doesn't make sense to try and guess what it might be. These values you probalby do want to keep as NaN. On the other hand, if a value is missing becuase it wasn't recorded, then you can try to guess what it might have been based on the other values in that column and row. (This is called "imputation" and we'll learn how to do it next! :)

Let's work through an example. Looking at the number of missing values in the nfl_data dataframe, I notice that the column `TimesSec` has a lot of missing values in it: 

In [ ]:
# look at the # of missing points in the first ten columns
missing_values_count[0:10]

By looking at [the documentation](https://www.kaggle.com/maxhorowitz/nflplaybyplay2009to2016), I can see that this column has information on the number of seconds left in the game when the play was made. This means that these values are probably missing because they were not recorded, rather than because they don't exist. So, it would make sense for us to try and guess what they should be rather than just leaving them as NA's.

On the other hand, there are other fields, like `PenalizedTeam` that also have lot of missing fields. In this case, though, the field is missing because if there was no penalty then it doesn't make sense to say *which* team was penalized. For this column, it would make more sense to either leave it empty or to add a third value like "neither" and use that to replace the NA's.

> **Tip:** This is a great place to read over the dataset documentation if you haven't already! If you're working with a dataset that you've gotten from another person, you can also try reaching out to them to get more information.

If you're doing very careful data analysis, this is the point at which you'd look at each column individually to figure out the best strategy for filling those missing values. For the rest of this notebook, we'll cover some "quick and dirty" techniques that can help you with missing values but will probably also end up removing some useful information or adding some noise to your data.

## Your turn!

* Look at the columns `Street Number Suffix` and `Zipcode` from the `sf_permits` datasets. Both of these contain missing values. Which, if either, of these are missing because they don't exist? Which, if either, are missing because they weren't recorded?

In [ ]:
# look at the # of missing points in the first ten columns of sf_permits
sf_missing_values_count[0:44]

# Drop missing values
___

If you're in a hurry or don't have a reason to figure out why your values are missing, one option you have is to just remove any rows or columns that contain missing values. (Note: I don't generally recommend this approch for important projects! It's usually worth it to take the time to go through your data and really look at all the columns with missing values one-by-one to really get to know your dataset.)  

If you're sure you want to drop rows with missing values, pandas does have a handy function, `dropna()` to help you do this. Let's try it out on our NFL dataset!

In [ ]:
# remove all the rows that contain a missing value
nfl_data.dropna()

In [ ]:
# remove all columns with at least one missing value
columns_with_na_dropped = nfl_data.dropna(axis=1)
columns_with_na_dropped.head()

In [ ]:
# just how much data did we lose?
print("Columns in original dataset: %d \n" % nfl_data.shape[1])
print("Columns with na's dropped: %d" % columns_with_na_dropped.shape[1])

## What Happened?
Oh dear, it looks like that's removed all our data! 😱 This is because every row in our dataset had at least one missing value. We might have better luck removing all the *columns* that have at least one missing value instead.

## What had happened was... (code explanation):
- `nfl_data.dropna()`: This line removes any row in the DataFrame nfl_data that contains at least one missing value.
- By default, `dropna()` operates along `axis=0` (rows), meaning if a row has any NaN (missing) values, it will be dropped.
- Important: This does not modify `nfl_data` in place unless you add `inplace=True`. Without `inplace=True`, the code simply returns a new DataFrame with the rows removed.
- `nfl_data.dropna(axis=1)`: This line removes any column that contains at least one missing value. Setting `axis=1` specifies that the operation should be applied to columns instead of rows.
- The result is stored in `columns_with_na_dropped`, which is a new DataFrame with all columns containing missing values removed.
- We've lost quite a bit of data, but at this point we have successfully removed all the `NaN`'s from our data. 


In [ ]:
# Your turn! Try removing all the rows from the sf_permits dataset that contain missing values. How many are left?
# remove all the rows that contain a missing value

In [ ]:
# Now try removing all the columns with empty values. Now how much of your data is left?
# remove all columns with at least one missing value

# Filling in missing values automatically
_____

Another option is to try and fill in the missing values. For this next bit, I'm getting a small sub-section of the NFL data so that it will print well.

In [ ]:
# get a small subset of the NFL dataset
subset_nfl_data = nfl_data.loc[:, 'EPA':'Season'].head()
subset_nfl_data

### Code Explanation: 
1. Column Selection (nfl_data.loc[:, 'EPA':'Season']):

    - `nfl_data.loc[:, 'EPA':'Season']`: This uses `.loc[]` to select a range of columns from 'EPA' to 'Season'.
    - The `:` before 'EPA' specifies that all rows should be included, and `'EPA':'Season'` specifies a range of columns, selecting from 'EPA' up to and including 'Season'.
2. Row Selection (`head()`):
    - `.head()`: This selects the first 5 rows of the DataFrame.
3. Storing the Subset:
    - The result, which is a small subset of `nfl_data` (the first 5 rows and columns from 'EPA' to 'Season'), is assigned to the variable `subset_nfl_data`.

Final Output
    - `subset_nfl_data` will contain a subset of `nfl_data`, limited to the first 5 rows and columns ranging from 'EPA' to 'Season'.
    
We can use the Panda's fillna() function to fill in missing values in a dataframe for us. One option we have is to specify what we want the `NaN` values to be replaced with. Here, I'm saying that I would like to replace all the `NaN` values with 0.

In [ ]:
# replace all NA's with 0
subset_nfl_data.fillna(0)

I could also be a bit more savvy and replace missing values with whatever value comes directly after it in the same column. (This makes a lot of sense for datasets where the observations have some sort of logical order to them.)

In [ ]:
# replace all NA's the value that comes directly after it in the same column, 
# then replace all the reamining na's with 0
subset_nfl_data.fillna(method = 'bfill', axis=0).fillna(0)

Filling in missing values is also known as "imputation", and you can find more exercises on it [in this lesson, also linked under the "More practice!" section](https://www.kaggle.com/dansbecker/handling-missing-values). 

First, however, why don't you try replacing some of the missing values in the sf_permit dataset? Let's identify a subset where we can make some assumptions.

How about the columns from `Plansets` to `Proposed Construction Type Description`?

In [ ]:

# get a small subset of the sf_permits dataset


In [ ]:

# replace all NA's with 0


In [ ]:

# replace all NA's with the value that two cells to the left, 
# then replace all the reamining na's with 0


# And that's it for today! 


## More practice!
___

If you're looking for more practice handling missing values, check out `Data Cleaning Challenge: Handling Missing Values`: https://www.kaggle.com/code/rtatman/data-cleaning-challenge-handling-missing-values

## Session 2 Exercise: Data Analysis and Visualization with NumPy, pandas, and Matplotlib

In this exercise, you will clean and analyze a dataset using NumPy, pandas, and matplotlib. This will allow you to:
- Perform some data cleaning on the dataset.
- Perform numerical operations with NumPy.
- Load, explore, and filter data using pandas.
- Visualize data trends with matplotlib.

The dataset provided is `city_temperature_with_nan.csv` with the following columns:
- **City**: Name of the city.
- **Date**: Date of the recorded temperature (YYYY-MM-DD).
- **Temperature**: Recorded temperature in degrees Celsius (some missing values).
- **Rainfall (mm)**: Recorded rainfall in millimeters (some missing values).
- **Time Zone**: Time zone city is located.

### Submission Instructions:
1. Rename your notebook in the following format: lastname_firstname.ipynb
2. Download your notebook from the dashboard to your local computer.
3. Email your notebook to egarza@tacc.utexas.edu, use subject line: ACSC Session 2 Deliverable

### Deadline to Submit
- If you don't plan to attend the training session then you deadline is 10:00p CT, 9:00p MT, 8:00p PT, 7:00p AKT, 5:00p HT Wednesday (10/30/24).
- For all others you can submit by 5:00p CT, 4:00p MT, 3:00p PT, 2:00p AKT, 12:00p HT Monday (11/04/24)
